In [88]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import requests
from selenium import webdriver
from datetime import date
import time
import dateutil.relativedelta

In [91]:
def getDownloadLink(data):
    link = []
    cont = 0
    while (not link or cont == 10):
        #Iniciando driver
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument("--window-size=800,800")
        driver = webdriver.Chrome(options=options)
        driver.get("https://portalibre.fgv.br/press-releases")
        
        #Selecionando Indice Geral de Preco
        select_element = driver.find_element(By.ID, 'edit-term-node-tid-depth-shs-0-0')
        select = Select(select_element)
        select.select_by_visible_text('Índice Geral de Preço')
        time.sleep(1)

        select_element = driver.find_element(By.ID, 'edit-term-node-tid-depth-shs-0-1')
        select = Select(select_element)
        select.select_by_visible_text('IGP-M')
        time.sleep(1)
        driver.find_element(By.ID, 'edit-submit-press-release').click()
        #button = driver.find_element(By.ID, 'edit-submit-press-release')
        #driver.execute_script("arguments[0].click();", button)
        time.sleep(2)
        elems = driver.find_elements(By.XPATH, '//a[contains(@href,".xls")]')
        links = []
        for elem in elems:
            links.append(elem.get_attribute("href"))
        mes_ano = str(data.strftime("%Y-%m"))
        link = [x for x in links if f'{mes_ano}' in x]
        cont+=1
        if not link:
            data = data - dateutil.relativedelta.relativedelta(months=1)
    return link[0]
    

In [84]:
def download_file(file_url,file_name):
    #Baixando o arquivo
    print(f"Downloading {file_url}")
    try:
        r = requests.get(file_url)
        open("bases_originais/"+file_name, 'wb').write(r.content)
    except Exception as e:
        print('Downloading file error: ', e)

In [93]:
def clean_df(file_name):
    print("Generating dataframe...")
    #Lendo dataframe
    df = pd.read_excel("bases_originais/"+file_name,header=[1])
    
    #Selecionando e renomeando colunas
    df = df[['Data', 'IGP-M', 'IGP-M.1']]\
    .rename(columns={"Data": "data", "IGP-M": "igp-mMensal", "IGP-M.1": "igp-m12m"})
    
    #formatando o dataframe
    df = df.drop(df[df["igp-mMensal"].str.contains('% mês', na=False)].index)\
    .drop(df[df["data"].str.contains('Fonte', na=False)].index)\
    .drop(df[df["data"].str.contains('Inicio da série', na=False)].index)
	
    #Arrumando tipagem dos dados
    df.data = df.data.astype(str)
    df["igp-mMensal"] = df["igp-mMensal"].astype(float)
    df["igp-m12m"] = df["igp-m12m"].astype(float)
    
    #Arrumando formato de data
    df['data'] = df['data'].apply(lambda x: r"1989-06-01 00:00:00" if x == r"jun/89 (*)" else x)
    
    return df

In [96]:
if __name__ == '__main__':
    file_name = "igpm.xls"
    file_url = getDownloadLink(date.today())
    download_file(file_url, file_name)
    df = clean_df(file_name)
    print(df)
    df.to_excel("bases_tratadas/"+file_name)

Generating dataframe...
                    data  igp-mMensal  igp-m12m
1    1989-06-01 00:00:00       0.1968       NaN
2    1989-07-01 00:00:00       0.3591       NaN
3    1989-08-01 00:00:00       0.3692       NaN
4    1989-09-01 00:00:00       0.3992       NaN
5    1989-10-01 00:00:00       0.4064       NaN
..                   ...          ...       ...
402  2022-11-01 00:00:00      -0.0056    0.0590
403  2022-12-01 00:00:00       0.0045    0.0545
404  2023-01-01 00:00:00       0.0021    0.0379
405  2023-02-01 00:00:00      -0.0006    0.0186
406  2023-03-01 00:00:00       0.0005    0.0017

[406 rows x 3 columns]


C:\Users\Contri\AppData\Local\Temp\ipykernel_25348\408139159.py:7: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel("bases_tratadas/"+file_name)


ModuleNotFoundError: No module named 'xlwt'